In [1]:
import os, pandas as pd

if os.path.exists("/kaggle"):
    DATA_DIR = "/kaggle/input/csiro-biomass"
    WORK_DIR = "/kaggle/working"
else:
    DATA_DIR = "~/kaggle/csiro/CSIRO-Image2Biomass/data"
    WORK_DIR = "~/kaggle/csiro/CSIRO-Image2Biomass/working"
    
test_long = pd.read_csv(os.path.join(DATA_DIR, "test.csv"))  # long format: one row per sample_id
sub = test_long[["sample_id"]].copy()
sub["target"] = 0.0                                    # trivial baseline
sub.to_csv(os.path.join(WORK_DIR, "submission.csv"), index=False)

print("Wrote submission.csv with", len(sub), "rows")


Wrote submission.csv with 5 rows
